# **ALPHA VOLATILITY GENERATION - SYSTEMATIC TRADING STRATEGIES PROJECT**

In [1]:
import numpy as np
from scipy.stats import norm
import pandas as pd
import importlib

In [2]:
from Data.market_data import Market_data
from backtester import Backtester

In [3]:
from strategy.base_strat import BaseStrategy
from strategy.regression_network import Regression_realvol, Regression_IV, Regression_IVvsRV

In [4]:
df_train = pd.read_pickle("df_train.pkl")
df_validation = pd.read_pickle("df_validation.pkl")
df_test = pd.read_pickle("df_test.pkl")
df_price = pd.read_pickle("df_price.pkl")
df_option = pd.read_pickle("df_merged.pkl")

In [5]:
data = Market_data(df_train,df_validation,df_test,df_price,df_option)

----

----

# **Regression network : Strategy based on a linear regression between implied volatility (IV) and realized volatility (RV)**

The strategy takes advantage of differences between implied volatility (IV) and realized volatility (RV) to find trading opportunities. For each option, the last 10 dates are considered, and a linear regression between IV and RV is performed to extract the slope, which reflects the recent trend. Here is the momentum strategy we have chosen **based on what worked best during the backtesting**  :

- If (the IV slope is above the 80th percentile of past slopes) **and** (the RV slope is below the 20th percentile), IV is overestimated compared to RV, and the strategy goes **LONG**.
- Conversely, if (the IV slope is below the 20th percentile) **and** (the RV slope is above the 80th percentile), IV is underestimated compared to RV, and the strategy goes **SHORT**.

The strategy exploits recent imbalances between implied and realized volatility, buying when the market overestimates volatility and selling when it underestimates it.


**DRAWBACK** : The main drawback of this strategy is that IV is aggregated using a simple daily average (by quote_date of the straddle). As a result, the strategy does not capture the term structure of volatility. The slope obtained from the regression over the last nb_period days only reflects the average IV trend, which is then compared to realized volatility (standard deviation of returns over the last nb_period days), calculated in the same way.

To sum up, the strategy does not take into account the term structure of straddles during the backtest. Moreover, the choice of quantiles was arbitrary, based on exploratory data analysis, and the window size (nb_period = 10) was also chosen in the same way.

In [6]:
regrv = Regression_realvol(data, 10)
regiv = Regression_IV(data, 10)
regivrv = Regression_IVvsRV(regiv, regrv, data, 10)

In [7]:
back = Backtester(data,regivrv)

In [8]:
print(back.run_backtest_IVvsRV()) # IV under RV over and IV over RV under
print(back.run_backtest_train())
print(back.run_backtest_validation())

The strategy achieved a success rate of 59.53% on 850 straddle trades in df_train.
The strategy achieved a success rate of 100.00% on 14 straddle trades in df_validation.
None
PNL:127.29262945552922, ROI:1.5535199632836612 %
None
PNL:26.74093148660205, ROI:23.681306665428668 %
None


------

-----

# **Strategy based on skew**